## Extractor: World Bank
Notebook used to extract information from the World Bank API; School Enrollment, primary and secondary (gross), gender parity index (GPI).  
  
To manually pull the data use: http://api.worldbank.org/v2/en/indicator/SE.ENR.PRSC.FM.ZS?downloadformat=csv

## 0 Dependencies
This Jupyter notebook is dependent on two resources:  
  
#### pandas_datareader
This library is used to query the World Bank API and convert the results into a DataFrame. This must be installed with the following on the BASH commandline: 'pip install pandas-datareader'  
  
#### secrets.py
Within the same folder as this jupyter notebook, the file "secrets.py" is required that contains a single entry for the password for the root user for the MySQL instance.  The entry is to be named "mysqlpassword"

In [1]:
#-- Import Libraries
import pandas as pd
from pandas_datareader import wb

from sqlalchemy import create_engine
import pymysql

pymysql.install_as_MySQLdb()

from secrets import mysqlpassword


#- Variables
rawDbName = 'SouthAmericaHappiness_Raw'

## 1 Extractor
Pull the data from the World Bank API.

### 1.1 Extractor Data from World Bank
Using the pandas_datareader that has functionality to pull from the World Bank API; pull data for the "School Enrollment, primary and secondary (gross), gender parity index (GPI)".  Get for all countries between the years of 2008 and 2018.

In [2]:
#- Set Incidicator
sourceIncidicator = 'SE.ENR.PRSC.FM.ZS'


#- Request Data from World Bank
genderParity_df = wb.download(indicator=sourceIncidicator, start=2008, end=2018, country='all')


#- Display Msg
print(f"Success in downloading indicator into dataframe. Count: {genderParity_df.count()}")

Success in downloading indicator into dataframe. Count: SE.ENR.PRSC.FM.ZS    1733
dtype: int64


### 1.2 Verify Data
Quickly check that the data returned from the endpoint contains the information requested.

In [3]:
#- Display Results
genderParity_df.head(25)

SE.ENR.PRSC.FM.ZS
country                        year                   
Arab World                     2018                NaN
                               2017            0.93583
                               2016            0.93506
                               2015            0.93472
                               2014            0.93097
                               2013            0.92959
                               2012            0.92904
                               2011            0.92285
                               2010            0.92433
                               2009            0.92310
                               2008            0.92215
Caribbean small states         2018                NaN
                               2017            1.02534
                               2016            1.02379
                               2015            1.04160
                               2014            1.02854
                               2013            1.02464
                               2012            1.03235
                               2011            1.03044
                               2010            1.02676
                               2009            1.01335
                               2008            1.02211
Central Europe and the Baltics 2018                NaN
                               2017            0.99432
                               2016            0.99418

### 1.3 Reset Index
The pandas_datareader library creates a hierarchical index based on the country.  This index is removed before being loaded into MySQL.  Blog post provides insight into this index: https://janakiev.com/blog/pandas-multiindex-pivot/

In [4]:
#- Create New Dataframe without index
genderParityData_df = genderParity_df.reset_index()


#- Display Results
genderParityData_df.head(25)

,country,year,SE.ENR.PRSC.FM.ZS
0,Arab World,2018,NaN
1,Arab World,2017,0.93583
2,Arab World,2016,0.93506
3,Arab World,2015,0.93472
4,Arab World,2014,0.93097
5,Arab World,2013,0.92959
6,Arab World,2012,0.92904
7,Arab World,2011,0.92285
8,Arab World,2010,0.92433
9,Arab World,2009,0.92310


## 2 Load
Load the data into RAW database.

### 2.1 Create Connection
Create a connection to the MySQL database; the password is located in the secrets.py.

In [5]:
#- Create Connection String
rds_connection_string = f"root:{mysqlpassword}@127.0.0.1/{rawDbName}"

#- Create Connection
engine = create_engine(f'mysql://{rds_connection_string}')

#- Display Message
print(f"Completed creating connection to database: {rawDbName}")

Completed creating connection to database: LatinAmericaHappiness_Raw


### 2.2 Load Data
Load the dataframe into the table within MySQL; replace any values in table if run additional times.

In [6]:
tableName = "WorldBank_SE.ENR.PRSC.FM.ZS"

genderParityData_df.to_sql(name= tableName,
                          con = engine,
                          if_exists='replace',
                          index= False)

print(f"Completed loading table with results; {tableName}")

Completed loading table with results; WorldBank_SE.ENR.PRSC.FM.ZS


## 3 Verify
Check to ensure that the records found within the DataFrame were all loaded successfully in the MySQL database.

In [7]:
#- Get Count of records in database
databaseCount = engine.scalar(f"SELECT COUNT(*) FROM `{tableName}`;")

print(f"Number of records found in database: {databaseCount}")

Number of records found in database: 2904


In [8]:
#- Get Count of records in DataFrame
genderParityData_df.shape

(2904, 3)

#### Results
The table within the database was found to have 2,904 records; the DataFrame had 2,904 records. Hence, the data was successfully loaded into the database.